In [1]:
# 首先加载必用的库
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import jieba # 结巴分词
# gensim用来加载预训练word vector
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings("ignore")
# 用来解压
import bz2

C:\Program Files\Anaconda3\envs\tensorflow_2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# 请将下载的词向量压缩包放置在根目录 embeddings 文件夹里
# 解压词向量, 有可能需要等待1-2分钟
# with open("embeddings/sgns.zhihu.bigram", 'wb') as new_file, open("embeddings/sgns.zhihu.bigram.bz2", 'rb') as file:
#     decompressor = bz2.BZ2Decompressor()
#     for data in iter(lambda : file.read(100 * 1024), b''):
#         new_file.write(decompressor.decompress(data))

In [3]:
# 使用gensim加载预训练中文分词embedding, 有可能需要等待1-2分钟
cn_model = KeyedVectors.load_word2vec_format('embeddings/sgns.zhihu.bigram', 
                                             binary=False, unicode_errors="ignore")

In [4]:
# 由此可见每一个词都对应一个长度为300的向量
embedding_dim = cn_model['山东大学'].shape[0]
print('词向量的长度为{}'.format(embedding_dim))
# cn_model['山东大学']

词向量的长度为300


In [5]:
# 计算相似度
# cn_model.similarity('橘子', '橙子')

In [6]:
# dot（'橘子'/|'橘子'|， '橙子'/|'橙子'| ）
# np.dot(cn_model['橘子']/np.linalg.norm(cn_model['橘子']), 
# cn_model['橙子']/np.linalg.norm(cn_model['橙子']))

In [7]:
# 找出最相近的词，余弦相似度
# cn_model.most_similar(positive=['大学'], topn=10)

In [5]:
# 我们使用tensorflow的keras接口来建模
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.python.keras.utils import to_categorical

In [69]:
# import numpy as np

In [6]:
train_texts_orig = []
# 文本所对应的labels, 也就是标记
train_target = []
f = open('评论/处理完/positive.csv',encoding='utf-8-sig')
dataset2 = pd.read_csv(f)
for i in range(len(dataset2)):
    train_texts_orig.append(dataset2['comment'][i])
    train_target.append(2)#dataset2['sentiment'][i]
f.close()
f = open('评论/处理完/negative.csv',encoding='utf-8-sig')
dataset0 = pd.read_csv(f)
for i in range(len(dataset0)):
    train_texts_orig.append(dataset0['comment'][i])
    train_target.append(0)#dataset0['sentiment'][i]
f.close()
f = open('评论/处理完/relevant.csv',encoding='utf-8-sig')
dataset1 = pd.read_csv(f)
for i in range(len(dataset1)):
    train_texts_orig.append(dataset1['comment'][i])
    train_target.append(1)#dataset0['sentiment'][i]
f.close()
# train_texts_orig=np.array(train_texts_orig)
train_target=to_categorical(train_target,num_classes=3) 
print(train_texts_orig)
print(train_target)

['找了好久', '同美术生', '听这首歌每次都会在脑海想象小丑女的模样', '魔性起飞', '这嗓音好上头 大哭 ', '“百因必有果，你的报应就是我” 韩美娟', '多听几遍，，，，你会get到的', '打破4892赞26回复可惜', '但是很喜欢', '希望世界安静 让所有情绪都见鬼.', '来自天堂的魔鬼 哈哈哈 ', '哈哈哈哈 大哭 ', '你也太狠了哈哈哈哈哈哈', '哈哈哈哈操，优秀优秀', '默默祝文刀小队长同志考研成功嘻嘻嘻', '多听几遍', '独特！', '独特，对头！ 强 强 强 ', '评论区是我今日的快乐源泉哈哈哈', '哈哈哈哈哈哈哈给爷整笑了', '满分', '小鱼人在唱歌，呜哇哇哇啊', '2分44的呐喊，鸡皮疙瘩起来了 多多捂脸 多多捂脸 多多捂脸 ', '这个歌手的嗓子跟这首很像 Black Sheep Gin Wigmore', '给你朋友一点时间，他就会，真香', '是个狼人', '这些和，越听越喜欢，真香', '我的天，今早英语课上老师放的就是这首歌，有毒吧 ', '加油', '还有被魔鬼舔过的。', '好听的像开玩笑唱的一样 西西心动 ', '各位好同学们 好！', '给 冲啊', '下次一定要在动物园放这首歌，让小猴子和我一起跳舞，，，，，，，，，，抖腿也行', '对我也想象了一下停不下来哈哈哈哈哈', '笑死我这个广西人哈哈哈哈哈哈', '第一次听 好过瘾第二次听 非常过瘾第三次听 超级过瘾 大哭 大哭 大哭 大哭 大哭 ', 'OIC to C to hit the wall a be the 多多笑哭 多多笑哭 多多笑哭 多多笑哭 ', 'What s how Lord how can you can put into fat', '吸毒吸毒吸毒啊 多多耍酷 多多耍酷 多多耍酷 多多耍酷 ', '首个太过瘾了一听就要条件反射，就想唱', '加油 ', '呼啊 笑死', '同一届！加油！', '哈哈哈哈哈哈这个歌有毒', '想听李荣浩那个好声音冠军唱这首歌', '听到这歌，每次脑海里都是菲尼克斯的小丑里，沿着台阶向下跳舞', '评论那会儿还没999 来着 哈哈哈', '好的', '哈哈哈', '加油 ', '英语曾老师给我们放了这首歌 喜欢她的美丽与魅力', '这首歌第一次分享给我朋友 他说好听 我觉得这个朋友

In [7]:
len(train_texts_orig)

57419

In [8]:
# 进行分词和tokenize
# train_tokens是一个长长的list，其中含有小list，对应每一条评价
train_tokens = []
for text in train_texts_orig:
    # 去掉标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 结巴分词
    cut = jieba.cut(text)
    # 结巴分词的输出结果为一个生成器
    # 把生成器转换为list
    cut_list = [ i for i in cut ]
    for i, word in enumerate(cut_list):
        try:
            # 将词转换为索引index
            cut_list[i] = cn_model.vocab[word].index
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list[i] = 0
    train_tokens.append(cut_list)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.819 seconds.
Prefix dict has been built succesfully.


In [9]:
# 获得所有tokens的长度
num_tokens = [ len(tokens) for tokens in train_tokens ]
num_tokens = np.array(num_tokens)

In [15]:
# np.min(num_tokens)

In [10]:
# 平均tokens的长度
np.mean(num_tokens)

11.638987094864069

In [11]:
# 最长的评价tokens的长度
np.max(num_tokens)

97

In [77]:
# np.log(num_tokens)

In [78]:
# plt.hist(np.log(num_tokens), bins = 1000)
# plt.xlim((0,10))
# plt.ylabel('number of tokens')
# plt.xlabel('length of tokens')
# plt.title('Distribution of tokens length')
# plt.show()

In [12]:
# 取tokens平均值并加上两个tokens的标准差，
# 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

39

In [13]:
# 取tokens的长度为39时，大约94%的样本被涵盖
# 我们对长度不足的进行padding，超长的进行修剪
np.sum( num_tokens < max_tokens ) / len(num_tokens)

0.9400198540552779

In [14]:
# 用来将tokens转换为文本
def reverse_tokens(tokens):
    text = ''
    for i in tokens:
        if i != 0:
            text = text + cn_model.index2word[i]
        else:
            text = text + ' '
    return text

In [82]:
# reverse = reverse_tokens(train_tokens[0])

In [83]:
# 经过tokenize再恢复成文本
# 可见标点符号都没有了
# reverse

In [84]:
# 原始文本
# train_texts_orig[0]

In [85]:
# embedding_dim

In [15]:
# 只使用前50000个词
num_words = 50000
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50000 * 300
for i in range(num_words):
    embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
embedding_matrix = embedding_matrix.astype('float32')

In [87]:
# 检查index是否对应，
# 输出300意义为长度为300的embedding向量一一对应
# np.sum( cn_model[cn_model.index2word[333]] == embedding_matrix[333] )

In [88]:
# embedding_matrix的维度，
# 这个维度为keras的要求，后续会在模型中用到
# embedding_matrix.shape

In [16]:
# 进行padding和truncating， 输入的train_tokens是一个list
# 返回的train_pad是一个numpy array
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')

In [17]:
# 超出五万个词向量的词用0代替
train_pad[ train_pad>=num_words ] = 0

In [18]:
# 可见padding之后前面的tokens全变成0，文本在最后面
train_pad[33]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  2694,   195,    20,    15,  7761,   468,
          22,  1954,    55, 45211,     5,     6,   192,  5831,  3292,
         844,    18,   792])

In [19]:
# 准备target向量，前2000样本为1，后2000为0
train_target = np.array(train_target)

In [20]:
# 进行训练和测试样本的分割
from sklearn.model_selection import train_test_split

In [21]:
# 90%的样本用来训练，剩余10%用来测试
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=12)

In [22]:
# 查看训练样本，确认无误
print(reverse_tokens(X_train[0]))
print('class: ',y_train[0])

                       人生建议别在不重要的人那里丢掉了你该有的快乐
class:  [1. 0. 0.]


In [23]:
# 用LSTM对样本进行分类
model = Sequential()

In [24]:
# 模型第一层为embedding
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))

In [25]:
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(LSTM(units=16, return_sequences=False))

In [26]:
model.add(Dense(3, activation='softmax'))#sigmoid
# 我们使用adam以0.01的learning rate进行优化
optimizer = Adam(lr=1e-2)

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [28]:
# 我们来看一下模型的结构，一共90k左右可训练的变量
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 39, 300)           15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 39, 128)           186880    
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense (Dense)                (None, 3)                 51        
Total params: 15,196,211
Trainable params: 196,211
Non-trainable params: 15,000,000
_________________________________________________________________


In [29]:
# 建立一个权重的存储点
path_checkpoint = 'sentiment_checkpoint_4.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [30]:
# 尝试加载已训练模型
try:
    model.load_weights(path_checkpoint)
except Exception as e:
    print(e)

# 保存模型

In [37]:
model.save('RNN_SentimentAnalysis_model_3.h5')

In [1]:
# from sklearn.externals import joblib
# # joblib.dump(model, 'RNN_SentimentAnalysis_model_3.pkl')
# print("Model dumped!")

Model dumped!


C:\Program Files\Anaconda3\envs\tensorflow_2\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [108]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [110]:
# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)

In [111]:
# 定义callback函数
callbacks = [
    earlystopping, 
    checkpoint,
    lr_reduction
]

In [112]:
# 开始训练
model.fit(X_train, y_train,
          validation_split=0.1, 
          epochs=20,
          batch_size=128,
          callbacks=callbacks)

Train on 46509 samples, validate on 5168 samples
Epoch 1/20
46464/46509 [============================>.] - ETA: 0s - loss: 0.4902 - accuracy: 0.8074
Epoch 00001: val_loss improved from 0.57406 to 0.57339, saving model to sentiment_checkpoint_4.keras
46509/46509 [==============================] - 72s 2ms/sample - loss: 0.4902 - accuracy: 0.8074 - val_loss: 0.5734 - val_accuracy: 0.7647
Epoch 2/20
46464/46509 [============================>.] - ETA: 0s - loss: 0.4880 - accuracy: 0.8087
Epoch 00002: val_loss improved from 0.57339 to 0.57297, saving model to sentiment_checkpoint_4.keras
46509/46509 [==============================] - 73s 2ms/sample - loss: 0.4881 - accuracy: 0.8087 - val_loss: 0.5730 - val_accuracy: 0.7647
Epoch 3/20
46464/46509 [============================>.] - ETA: 0s - loss: 0.4862 - accuracy: 0.8097
Epoch 00003: val_loss improved from 0.57297 to 0.57264, saving model to sentiment_checkpoint_4.keras
46509/46509 [==============================] - 71s 2ms/sample - loss: 0.

In [2]:
# data=input_transform('魔性起飞')
# data.reshape(1,-1)
# model.predict_classes(data)

In [1]:
# train_target

In [105]:
# def predict_sentiment(text):
#     print(text)
#     # 去标点
#     text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
#     # 分词
#     cut = jieba.cut(text)
#     cut_list = [ i for i in cut ]
#     # tokenize
#     for i, word in enumerate(cut_list):
#         try:
#             cut_list[i] = cn_model.vocab[word].index
#             if cut_list[i] >= 50000:
#                 cut_list[i] = 0
#         except KeyError:
#             cut_list[i] = 0
#     # padding
#     tokens_pad = pad_sequences([cut_list], maxlen=max_tokens,
#                            padding='pre', truncating='pre')
#     # 预测
#     result = model.predict(x=tokens_pad)
#     coef = result[0][0]
#     if coef >= 0.5:
#         print('是一例正面评价','output=%.2f'%coef)
#     else:
#         print('是一例负面评价','output=%.2f'%coef)

In [106]:
# predict_sentiment('魔性起飞')

魔性起飞
是一例正面评价 output=0.99
